## Query Decomposition: Avoid semantic delutions in the queries
 * "Semantic dilutions" refer to the loss of meaning or relevance when chunks of text, intended to represent information for retrieval, become too large or lack semantic cohesion, leading to less accurate and useful results.
 * Semantic dilutions can be mitigated on the indexed documents side by applying optimized chunking strategies per each use case.
 * Query texts can also have semantic delutions when multiple contexts or topics are combined. This can be improved by reforming the queries.

This notebook will demonstrate "query decomposition" strategy to improve the search quality against a knowledge base. 

![Query Decomposition](https://raw.githubusercontent.com/aws-samples/langgraph-agents-with-amazon-bedrock/refs/heads/main/assets/lab3_2.png "https://github.com/aws-samples/langgraph-agents-with-amazon-bedrock/tree/main/Lab_3")

#### Prerequisites

In [ ]:

import importlib
import advanced_rag_utils

# Reload module
importlib.reload(advanced_rag_utils)

# Re-import all functions
from advanced_rag_utils import *

from datetime import datetime, timedelta, UTC

notebook_start_time = datetime.now(UTC)
suppress_warnings()


In [ ]:
# Load variables from the JSON file
import json
with open("../variables.json", "r") as f:
    variables = json.load(f)

variables

In [ ]:
# Get KB configuration
kb_config = get_kb_config(
    kb_id=variables["kbSemanticChunk"],
    account_number=variables['accountNumber'],
    region_name=variables['regionName'],
    number_of_results=5
)

### How Model Size Affects Table Interpretation  

When querying Amazon's Operating Income for 2022, **smaller models (Nova Lite, Llama 3B)** tend to pick the **"At Prior Year Rates"** value (\$11,387), while **larger models (Nova Pro)** correctly select the **"As Reported"** value (\$12,248).

#### Possible Reasons:
- **Table Parsing Limitations:** Smaller models may not accurately align column headers to values.  
- **Context Misinterpretation:** They might default to the last numerical column or fail to strongly associate **"As Reported"** with the correct column.  
- **Stronger Reasoning in Larger Models:** Nova Pro better understands structured data, leading to more accurate retrieval.  

![Image](./operating_income.png)

### Basic RAG query
The query retrieves only one of the topics mentioned. It is because of the mix of two queries dilluted the second topic, which is called "semantic dillution."
In real world scenarios, it is required to pull multiple chunks with different contexts, for example:
* How Amazon's net income increased from 2018 to 2024?
* What is difference between RAG and text-to-SQL?

In [ ]:
# WITHOUT QUERY DECOMPOSITION

# Query example
query = "What was Amazon's Operating Income as reported for the fiscal year ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Perform basic RAG without query decomposition
response = retrieve_and_generate_basic(
    query=query, 
    kb_id=kb_config["kb_id"], 
    model_id=kb_config["model_id"], 
    generation_configuration=kb_config["generation_configuration"],
    number_of_results=kb_config["number_of_results"],
    region_name=variables["regionName"]
)

# Display the results
display_rag_results(response)

### Use Amazon Bedrock API feature to decompose a query.
ReteriveAndGenerate API support built-in query decompose feature.

In [ ]:
# WITH QUERY DECOMPOSITION

# Use Nova Pro model for better table interpretation
nova_pro_model_id = f"arn:aws:bedrock:us-west-2:{variables['accountNumber']}:inference-profile/us.amazon.nova-pro-v1:0"

# Query example (same as before)
query = "What was Amazon's Operating Income as reported for the fiscal year ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Perform RAG with query decomposition
response = retrieve_and_generate_with_decomposition(
    query=query, 
    kb_id=kb_config["kb_id"], 
    model_id=nova_pro_model_id, 
    generation_configuration=kb_config["generation_configuration"],
    number_of_results=kb_config["number_of_results"],
    region_name=variables["regionName"]
)

# Display the results
display_rag_results(response)

## Query Decomposition with Agentic RAG using SageMaker and LangChain

#### Prerequisites

In [ ]:
# Set up LangChain components
llm, retriever = setup_langchain_components(
    endpoint_name=variables["sagemakerLLMEndpoint"],
    kb_id=variables["kbSemanticChunk"],
    region_name=variables["regionName"],
    number_of_results=5
)

#### Test a complex query with plain Q&A chain

In [ ]:
# Create a simple QA chain
qa_chain = create_qa_chain(retriever, llm)

In [ ]:
# Query example
query = "What was the Operating Income of Amazon As Reported for the Year Ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Invoke the QA chain
answer = qa_chain.invoke(query)

print("Answer:", answer)

### Query Decomposition using Agentic RAG with LangChain

In [ ]:
# Create a custom output parser
custom_parser = create_output_parser()

In [ ]:
# Set up agentic RAG with query decomposition
agent = setup_agentic_rag(retriever, llm)

# Query example
query = "What was Amazon's Operating Income as reported for the fiscal year ending December 31, 2022? What is text-to-SQL? How did text-to-SQL contribute to Amazons earnings, if any?"

# Run the agent
result = agent.run(query)
print(result)